<a href="https://colab.research.google.com/github/ArmFriiz/Dicoding-Submission-FDL/blob/main/Analisis%20Sentimen/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Load Package dan Library**

In [118]:
!pip install gensim

In [119]:
import pandas as pd
import numpy as np
import tensorflow as tf
import re
import string
import nltk
import kagglehub
import gensim

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

In [121]:
path = kagglehub.dataset_download("bhimantoros/pretrained-word2vec-indonesia")
print("Path to dataset files:", path)

Using Colab cache for faster access to the 'pretrained-word2vec-indonesia' dataset.
Path to dataset files: /kaggle/input/pretrained-word2vec-indonesia


**Load Dataset Sentimen**

In [122]:
df = pd.read_csv('https://raw.githubusercontent.com/ArmFriiz/Dicoding-Submission-FDL/refs/heads/main/Analisis%20Sentimen/dataset_ulasan_playstore.csv')

In [123]:
df.head(5)

,content,score,label
0,CEPAT AMAN TERPERCAYA,5,Positif
1,top markotop,5,Positif
2,belanja online nomor 1 satu di Indonesia.,5,Positif
3,GANGGU DOANG LU ORG LAGI MAIN JUGA,1,Negatif
4,jadi langganan belanja di shopee..,5,Positif


In [124]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73478 entries, 0 to 73477
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   content  73478 non-null  object
 1   score    73478 non-null  int64 
 2   label    73478 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.7+ MB


In [125]:
df.describe(include='all')

,content,score,label
count,73478,73478.000000,73478
unique,72705,NaN,3
top,Baik,NaN,Positif
freq,5,NaN,51173
mean,NaN,3.862802,NaN
std,NaN,1.668418,NaN
min,NaN,1.000000,NaN
25%,NaN,2.000000,NaN
50%,NaN,5.000000,NaN
75%,NaN,5.000000,NaN


In [126]:
df.drop(columns=['score'], inplace=True)

In [127]:
df.columns

Index(['content', 'label'], dtype='object')

**Load Model Normalisasi Kata**

In [128]:
normalization_df = pd.read_csv('https://raw.githubusercontent.com/nasalsabila/kamus-alay/refs/heads/master/colloquial-indonesian-lexicon.csv')

In [129]:
normalization_df.head()

,slang,formal,In-dictionary,context,category1,category2,category3
0,woww,wow,1,wow,elongasi,0,0
1,aminn,amin,1,Selamat ulang tahun kakak tulus semoga panjang...,elongasi,0,0
2,met,selamat,1,Met hari netaas kak!? Wish you all the best @t...,abreviasi,0,0
3,netaas,menetas,1,Met hari netaas kak!? Wish you all the best @t...,afiksasi,elongasi,0
4,keberpa,keberapa,0,Birthday yg keberpa kak?,abreviasi,0,0


In [130]:
normalization_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15006 entries, 0 to 15005
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   slang          15006 non-null  object
 1   formal         15006 non-null  object
 2   In-dictionary  15006 non-null  int64 
 3   context        15006 non-null  object
 4   category1      15006 non-null  object
 5   category2      15006 non-null  object
 6   category3      15006 non-null  object
dtypes: int64(1), object(6)
memory usage: 820.8+ KB


In [131]:
normalization_dict = dict(zip(normalization_df['slang'], normalization_df['formal']))

**Load Model Word2Vec**

In [132]:
w2v_model = gensim.models.Word2Vec.load("/root/.cache/kagglehub/datasets/bhimantoros/pretrained-word2vec-indonesia/versions/1/wiki.id.case.model").wv

**Function Cleaning**

In [133]:
def cleaning_text(text, normalization_dict):
  text = text.lower() # Ubah teks menjadi huruf kecil
  text = re.sub(r'@[A-Za-z0-9]+', '', text) # Hapus mention
  text = re.sub(r'http\S+', '', text) # Hapus URL
  text = re.sub(r'#[A-Za-z0-9]+', '', text) # Hapus hashtag
  text = re.sub(r'(.)\1{2,}', r'\1\1', text) # Hapus kata berulang yang muncul lebih dari 2x

  words = text.split()
  words = [normalization_dict.get(word, word) for word in words]
  text = ' '.join(words)

  text = ''.join([char for char in text if char.isalpha() or char.isspace()])
  text = ' '.join(text.split()) # Bersihkan spasi berlebih di tengah dan di ujung
  return text

In [134]:
df['clean_content'] = df['content'].apply(cleaning_text, normalization_dict=normalization_dict)
# df['tokenized_content'] = df['clean_content'].apply(tokenizingText)
# df['filtered_content'] = df['tokenized_content'].apply(filteringText)
# df['stemmed_content'] = df['filtered_content'].apply(lambda x: stemmingText(x, stemmer))

In [135]:
df

,content,label,clean_content
0,CEPAT AMAN TERPERCAYA,Positif,cepat aman terpercaya
1,top markotop,Positif,top markotop
2,belanja online nomor 1 satu di Indonesia.,Positif,belanja online nomor satu di indonesia
3,GANGGU DOANG LU ORG LAGI MAIN JUGA,Negatif,ganggu doang lu orang lagi main juga
4,jadi langganan belanja di shopee..,Positif,jadi langganan belanja di shopee
...,...,...,...
73473,murah pokoknu beli di shopee,Positif,murah pokoknu beli di shopee
73474,"selalu puas belanja di shopee, mudah di aplika...",Positif,selalu puas belanja di shopee mudah di aplikas...
73475,"berbelanja murah, pengiriman cepat dan pelayan...",Positif,berbelanja murah pengiriman cepat dan pelayana...
73476,"sejak shopee indonesia ada, yg namanya outfit,...",Positif,sejak shopee indonesia ada yang namanya outfit...


In [136]:
# top_20 = df['tokenized_content'].explode().value_counts().head(20)
# top_20